In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/meatconsumption/meat_consumption_worldwide.csv
/kaggle/input/countries-code-and-continent/all.csv


In [2]:
df= pd.read_csv("/kaggle/input/meatconsumption/meat_consumption_worldwide.csv")
country_df= pd.read_csv("/kaggle/input/countries-code-and-continent/all.csv")

**Data Cleaning and Gathering**:
* This dataset have 5 columns: location (countries) , subject (type of meat) , measurement, time (years), value (amount of meats)
* This dataset have 2 different measurements: **KG_CAP** (kg per capital- retail weight) & **THND_TONNE** (thousand tonnes - carcasses weight). Therefore, I will just use KG_CAP as my primary measurement.
* Timeline: 1990 to 2026

**Task** 
1. What country consume the most and least meat?
2. What continent consume the most and least meat?
3. Overall meat consumption break down 
4. Timeline of meat consumption

In [3]:
df.head()

,LOCATION,SUBJECT,MEASURE,TIME,Value
0,AUS,BEEF,KG_CAP,1991,27.721815
1,AUS,BEEF,KG_CAP,1992,26.199591
2,AUS,BEEF,KG_CAP,1993,26.169094
3,AUS,BEEF,KG_CAP,1994,25.456134
4,AUS,BEEF,KG_CAP,1995,25.340226


In [4]:
clean_df = df[df['MEASURE']=="KG_CAP"] # filter to only have kg_cap
clean_df

,LOCATION,SUBJECT,MEASURE,TIME,Value
0,AUS,BEEF,KG_CAP,1991,27.721815
1,AUS,BEEF,KG_CAP,1992,26.199591
2,AUS,BEEF,KG_CAP,1993,26.169094
3,AUS,BEEF,KG_CAP,1994,25.456134
4,AUS,BEEF,KG_CAP,1995,25.340226
...,...,...,...,...,...
6713,EU28,SHEEP,KG_CAP,2022,2.012270
6714,EU28,SHEEP,KG_CAP,2023,2.018727
6715,EU28,SHEEP,KG_CAP,2024,2.026332
6716,EU28,SHEEP,KG_CAP,2025,2.034105


In [5]:
clean_df = clean_df.rename(columns={'Value': 'VALUE'})
clean_df.head(5)

,LOCATION,SUBJECT,MEASURE,TIME,VALUE
0,AUS,BEEF,KG_CAP,1991,27.721815
1,AUS,BEEF,KG_CAP,1992,26.199591
2,AUS,BEEF,KG_CAP,1993,26.169094
3,AUS,BEEF,KG_CAP,1994,25.456134
4,AUS,BEEF,KG_CAP,1995,25.340226


In [6]:
clean_df.info() # no null

<class 'pandas.core.frame.DataFrame'>
Index: 6718 entries, 0 to 6717
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   LOCATION  6718 non-null   object 
 1   SUBJECT   6718 non-null   object 
 2   MEASURE   6718 non-null   object 
 3   TIME      6718 non-null   int64  
 4   VALUE     6718 non-null   float64
dtypes: float64(1), int64(1), object(3)
memory usage: 314.9+ KB


In [7]:
clean_df.duplicated().sum()

0

In [8]:
clean_df.describe()

,TIME,VALUE
count,6718.000000,6.718000e+03
mean,2008.215540,9.173819e+00
std,10.580345,1.114479e+01
min,1990.000000,1.669000e-09
25%,1999.000000,1.117657e+00
50%,2008.000000,4.246738e+00
75%,2017.000000,1.355567e+01
max,2026.000000,6.605042e+01


In [9]:
clean_df['LOCATION'].unique() 
#MeatDF['LOCATION'].nunique() #48 countries

array(['AUS', 'CAN', 'JPN', 'KOR', 'MEX', 'NZL', 'TUR', 'USA', 'DZA',
       'ARG', 'BGD', 'BRA', 'CHL', 'CHN', 'COL', 'EGY', 'ETH', 'GHA',
       'HTI', 'IND', 'IDN', 'IRN', 'ISR', 'KAZ', 'MYS', 'MOZ', 'NGA',
       'PAK', 'PRY', 'PER', 'PHL', 'RUS', 'SAU', 'ZAF', 'SDN', 'TZA',
       'THA', 'UKR', 'URY', 'VNM', 'ZMB', 'WLD', 'SSA', 'OECD', 'BRICS',
       'EU28'], dtype=object)

What country consume the most amount of meat?

In [10]:
meat_cs = clean_df.groupby(['LOCATION']).sum()['VALUE'].reset_index().sort_values('VALUE',ascending=False)
#meat_cs

In [11]:
px.bar(meat_cs,meat_cs['LOCATION'],meat_cs['VALUE'])

In [12]:
time_meat_cs = clean_df.groupby(['TIME','SUBJECT']).sum()['VALUE'].reset_index().sort_values('VALUE',ascending=False)
time_meat_cs

,TIME,SUBJECT,VALUE
146,2026,POULTRY,922.280888
142,2025,POULTRY,918.166830
138,2024,POULTRY,913.777554
134,2023,POULTRY,909.058695
130,2022,POULTRY,904.708187
...,...,...,...
99,2014,SHEEP,84.366721
107,2016,SHEEP,83.308705
95,2013,SHEEP,81.331912
79,2009,SHEEP,80.770468


In [13]:
px.bar(time_meat_cs,time_meat_cs['TIME'],time_meat_cs['VALUE'],time_meat_cs['SUBJECT'])#text_auto= True)

In [14]:
subject_meat_cs = clean_df.groupby('SUBJECT').sum()['VALUE'].reset_index().sort_values('VALUE',ascending=False)
subject_meat_cs

,SUBJECT,VALUE
2,POULTRY,26378.715109
0,BEEF,16742.934900
1,PIG,14750.652562
3,SHEEP,3757.415149


In [15]:
fig = px.pie(subject_meat_cs, values= 'VALUE', names="SUBJECT")
fig.show()

In [16]:
country_df.head(5)

,name,alpha-2,alpha-3,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code
0,Afghanistan,AF,AFG,4,ISO 3166-2:AF,Asia,Southern Asia,NaN,142.0,34.0,NaN
1,Åland Islands,AX,ALA,248,ISO 3166-2:AX,Europe,Northern Europe,NaN,150.0,154.0,NaN
2,Albania,AL,ALB,8,ISO 3166-2:AL,Europe,Southern Europe,NaN,150.0,39.0,NaN
3,Algeria,DZ,DZA,12,ISO 3166-2:DZ,Africa,Northern Africa,NaN,2.0,15.0,NaN
4,American Samoa,AS,ASM,16,ISO 3166-2:AS,Oceania,Polynesia,NaN,9.0,61.0,NaN


In [17]:
country_df = country_df .drop(['alpha-2', 'country-code','iso_3166-2','sub-region','intermediate-region','region-code','sub-region-code','intermediate-region-code'], axis=1)

In [18]:
country_df

,name,alpha-3,region
0,Afghanistan,AFG,Asia
1,Åland Islands,ALA,Europe
2,Albania,ALB,Europe
3,Algeria,DZA,Africa
4,American Samoa,ASM,Oceania
...,...,...,...
244,Wallis and Futuna,WLF,Oceania
245,Western Sahara,ESH,Africa
246,Yemen,YEM,Asia
247,Zambia,ZMB,Africa


In [19]:
df_concatenated = pd.concat([clean_df,country_df], axis=1)
df_concatenated

,LOCATION,SUBJECT,MEASURE,TIME,VALUE,name,alpha-3,region
0,AUS,BEEF,KG_CAP,1991,27.721815,Afghanistan,AFG,Asia
1,AUS,BEEF,KG_CAP,1992,26.199591,Åland Islands,ALA,Europe
2,AUS,BEEF,KG_CAP,1993,26.169094,Albania,ALB,Europe
3,AUS,BEEF,KG_CAP,1994,25.456134,Algeria,DZA,Africa
4,AUS,BEEF,KG_CAP,1995,25.340226,American Samoa,ASM,Oceania
...,...,...,...,...,...,...,...,...
6713,EU28,SHEEP,KG_CAP,2022,2.012270,NaN,NaN,NaN
6714,EU28,SHEEP,KG_CAP,2023,2.018727,NaN,NaN,NaN
6715,EU28,SHEEP,KG_CAP,2024,2.026332,NaN,NaN,NaN
6716,EU28,SHEEP,KG_CAP,2025,2.034105,NaN,NaN,NaN


Filter: 'WLD', 'SSA', 'OECD', 'BRICS', 'EU28'

In [20]:
no_nans = df_concatenated[~df_concatenated.isnull().any(axis=1)]
no_nans

,LOCATION,SUBJECT,MEASURE,TIME,VALUE,name,alpha-3,region
0,AUS,BEEF,KG_CAP,1991,27.721815,Afghanistan,AFG,Asia
1,AUS,BEEF,KG_CAP,1992,26.199591,Åland Islands,ALA,Europe
2,AUS,BEEF,KG_CAP,1993,26.169094,Albania,ALB,Europe
3,AUS,BEEF,KG_CAP,1994,25.456134,Algeria,DZA,Africa
4,AUS,BEEF,KG_CAP,1995,25.340226,American Samoa,ASM,Oceania
...,...,...,...,...,...,...,...,...
244,CAN,POULTRY,KG_CAP,2014,32.686144,Wallis and Futuna,WLF,Oceania
245,CAN,POULTRY,KG_CAP,2015,34.106558,Western Sahara,ESH,Africa
246,CAN,POULTRY,KG_CAP,2016,34.816802,Yemen,YEM,Asia
247,CAN,POULTRY,KG_CAP,2017,34.891265,Zambia,ZMB,Africa


In [21]:
continent_meat_cs= no_nans.groupby(['region','SUBJECT'])['VALUE'].sum().sort_values(ascending=False).reset_index()
continent_meat_cs

,region,SUBJECT,VALUE
0,Americas,POULTRY,528.192317
1,Asia,POULTRY,499.918540
2,Europe,POULTRY,491.934330
3,Africa,PIG,477.995256
4,Americas,BEEF,395.507310
5,Americas,PIG,385.367298
6,Africa,POULTRY,381.080676
7,Europe,BEEF,360.620797
8,Asia,BEEF,333.551891
9,Oceania,POULTRY,305.577326


In [22]:
px.bar(continent_meat_cs,continent_meat_cs['region'],continent_meat_cs['VALUE'],continent_meat_cs['SUBJECT'])

In [23]:
fig1 = px.pie(continent_meat_cs, values= 'VALUE', names="region")
fig1.show()